# COMPSCI 690V - hw1 - Interactive Bokeh Visualizations

### Read in three datasets from the Gapminder website.

In [8]:
import pandas as pd

gdp_df = pd.read_excel('data/Gapminder/indicator gapminder gdp_per_capita_ppp.xlsx', index_col=0)
traffic_df = pd.read_excel('data/Gapminder/RTI age adjusted indicator LIVE.xlsx', index_col=0)
pop_growth_df = pd.read_excel('data/Gapminder/population growth.xlsx', index_col=0)


# Make the column names ints not strings for handling.
columns = list(pop_growth_df.columns)
years_to_int = list(range(int(pop_growth_df.columns[0]), int(pop_growth_df.columns[-1])+1))
rename_dict = dict(zip(columns, years_to_int))
pop_growth_df = pop_growth_df.rename(columns=rename_dict)


# Find the intersection of countries and years among the three datasets.
countries = list(set(gdp_df.index).intersection(traffic_df.index, pop_growth_df.index))
years = list(set(gdp_df.columns).intersection(traffic_df.columns, pop_growth_df.columns))


# Get the clean data.
gdp_df = gdp_df[years].loc[countries]
traffic_df = traffic_df[years].loc[countries]
pop_growth_df = pop_growth_df[years].loc[countries]


data={}
panel = pd.Panel({'gdp': gdp_df, 'traffic': traffic_df, 'pop': pop_growth_df})
for year in years:
    df = panel[:,:,year]
    df['countries'] = countries
    data[year] = df.to_dict('series')

### Import standard functions.

In [9]:
from bokeh.io import output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.models import (
    ColumnDataSource, SingleIntervalTicker, CustomJS, Slider,
    TapTool, HoverTool, BoxSelectTool, BoxZoomTool, LassoSelectTool
)
from bokeh.models import NumeralTickFormatter
from ipywidgets import interact
from bokeh.layouts import column, row, gridplot

output_notebook()

Loading BokehJS ...

### Plot the data using bokeh library.

In [15]:
year = years[-3]

s1 = ColumnDataSource(data=data[year])
s2 = ColumnDataSource(data=dict(x=[], y=[]))
s3 = ColumnDataSource(data=dict(x=[], y=[]))
gdp_source = ColumnDataSource(gdp_df.transpose())

TOOLS_left = "pan,box_select,box_zoom,lasso_select,reset"
TOOLS_right = "pan,box_select,box_zoom,reset"

left = figure(
    plot_width=500,
    plot_height=500,
    tools=TOOLS_left,
    title="Scatter Plot",
    toolbar_location="above"
)

left.xaxis.ticker = SingleIntervalTicker(interval=20000)
left.yaxis.ticker = SingleIntervalTicker(interval=5)
left.xaxis[0].formatter = NumeralTickFormatter(format="$0,0")
left.yaxis[0].formatter = NumeralTickFormatter(format="0")
left.xaxis.axis_label = 'GDP per capita'
left.yaxis.axis_label = 'Traffic mortality per 100,000, age adjusted'

left.circle(
    'gdp',
    'traffic',
    source=s1,
    size=20,
    fill_alpha=0.4,
    line_alpha=0.4,
    fill_color='olive',
    line_color='olive',
    
    hover_fill_color="firebrick",
    hover_line_color="firebrick",
    hover_fill_alpha=0.8,
    
    # set visual properties for selected glyphs
    selection_color="firebrick",
    selection_line_color="firebrick",
    selection_fill_alpha=0.8,
)

right_up = figure(
    plot_width=450,
    plot_height=250,
    tools=TOOLS_right,
    title="GDP Time Series",
    toolbar_location="above"
)
right_up.line('x', 'y', source=s2, alpha=0.6)

right_down = figure(
    plot_width=450,
    plot_height=250,
    tools=TOOLS_right,
    title="Traffic Mortality Time Series",
    toolbar_location="above"
)
right_down.line('x', 'y', source=s3, alpha=0.6)

hover = HoverTool(
        tooltips=[
            ("Country", "@countries"),
            ("GDP", "@gdp{$ 0,0}"),
            ("Traffic mortality", "@traffic{0.0}"),
            ("Population growth", "@pop{0.00}")
        ]
    )

tap = TapTool(callback=CustomJS(args=dict(s1=s1), code="""
    alert(1);
    var selected= cb_obj.selected['1d'].indices;
    var data = cb_obj.data;
    var country = data['countries'][selected];
    alert(selected)
    """))

left.add_tools(hover)
left.add_tools(tap)

def update(year):
    new_data = data[year]
    s1.data = new_data
    left.title.text = str(year)
    push_notebook()


plot= row(left, column(right_up, right_down))

show(plot, notebook_handle=True);
interact(update, year=(years[0], years[-1], 1));